# Table of Contents


## Link prediction
Link prediction can refer to either inferring links that are likely to appear in a dynamic network or missing links in a static network <a name="ref-1"/>[(Lü and Zhou, 2011)](#cite-Lu20111150). In a network of nodes and links $G(V,E)$ a prediction can return a ranking of all possible non-existing links $U-E$ where $U$ is the universal set consisting of all posible links between 
nodes. The ranking itself is found by sorting the potential links by some method dependent score.

To compare the  methods the benchmarks used are area under the ROC curve and precision along with a training and validation set, $U^T$ and $U^P$. It is standard to run the algorithms on the full set of $U-E^T$ <a name="ref-2"/>[(Lü and Zhou, 2011)](#cite-Lu20111150), but for computation time reasons it might be possible to only select a subset of $U-E$ as non-existent links. The area under the ROC (AUC) is found by comparing the scores of every non-existent link with the score of a removed link. 

$$
AUC = \frac{n'+0.5n''}{n}
$$

where $n'$ is the number of trials where the removed link had a higher score than the non-existent one and the $n''$ is the number of times the score of the non-existent link was greater or equal to the score of the removed link. $n$ is the total number of trials.

For precision the top $L$ ranked links are found where precision will be

$$
p = \frac{|L'|}{|L|}
$$

where $L'$ is the true positives.

The methods applied in link prediction can be divided into those that are based only on network structure and those that take into account the properties of the nodes to generate potential new links and the structural methods have further subdivisions into similarity-based algorithms and those that are based on probabilistic methods <a name="ref-3"/>[(Lü and Zhou, 2011)](#cite-Lu20111150). For now I will only look at similarity based structural methods.

### Similarity based methods

The similarity based link prediction methods can be further subdivided based on on how large a part of the network they take into account when scoring a potential link, leading to local methods, global methods and quasi-local methods <a name="ref-4"/>[(Lü and Zhou, 2011)](#cite-Lu20111150). Local indexing only looks at the immediate neighbors of a node, global methods look at the entire network and quasi-local are local but can extend beyond the 1-neighborhood.

Tables of several different similarity based methods can be found below with short explanations of their functionality. Here two nodes are designated $x$ and $y$, their 1-neighborhood is returned by the $\Gamma$ function and the degree of node $i$ is $k_i$.

### Local indexing
Similarity based methods with local indexing only look at the 1-neighborhood of any given node making them fast to compute but potentially losing information found in extended neighborhood. The table below shows a summary of each method along with averaged AUC and precision scores computed for a 5-folded cross validation.

| Name  | Definition | Comment |  Precison | AUC |
|---|------------|---|
| Common Neighbors  | $s_{x,y} = \Gamma(x) \cap \Gamma(y)$ | Based on common amount of neighbours between node x and node y | 0.608 | 0.730 |
| Salton Index      | $s_{x,y} = \frac{\Gamma(x) \cap \Gamma(y)}{\sqrt{k_x \times k_y}}$ | Comparable to the cosine index between nodes x and y | 0.668 | 0.728 |
| Jaccard Index     | $s_{x,y} = \frac{\Gamma(x) \cap \Gamma(y)}{\Gamma(x) \cup \Gamma(y)}$ | Normalizes the number of common neighbors with the total potential number of common neighbors. Equal to the probability of a randomly selected link belonging to both nodes | 0.668 | 0.728 |
| Adamic / Adar | $s_{x,y} = \sum_{z \ \in \ \Gamma(x) \cap \Gamma(y)} \frac{1}{\log k_z}$ | Assigns less weight to highly connected nodes, reducing the effect of hubs | 0.616 | 0.730 |
| Preferential Attachment | $s_{x,y} = k_x \times k_y$ | Probability of generating links is proportional only to the node degree. Low complexity, but does not take into account the cost of creating links in for example physical networks. | 0.564 | 0.674 |
| Resource Allocation | $s_{x,y} = \sum_{ z \ \in \ \Gamma(x) \cap \Gamma(y)} \frac{1}{k_z}$ | Based on $x$ diffusing a resource to $y$, with each neighboring being a channel | 0.648 | 0.731 |
| Leicht, Holme, Newman (LHN1) | $s_{x,y} = \frac{\Gamma(x) \cap \Gamma(y)}{k_x \times k_y}$ | Common neighbors proportional to the mean degree of nodes | 0.744 | 0.507 |
| Hub promoted index | $s_{x,y} = \frac{\Gamma(x) \cap \Gamma(y)}{\min(k_x, k_y)}$ | Score that amplifies the importance of hub connections | 0.524 | 0.502 |
| Hub depressed index | $s_{x,y} = \frac{\Gamma(x) \cap \Gamma(y)}{\max(k_x, k_y)}$ | Score that penalizes hub connections | 0.544 | 0.510 |
| Sørensen index | $s_{x,y} = \frac{2 \Gamma(x) \cap \Gamma(y)}{k_x + k_y}$ | Score used in ecological community data | 0.520 | 0.502 |

### Global indexing

Global indexing refers to methods that take into account the entire network which often leads to closed form solutions including matrix inversions. Since these have a compute time of $\mathcal{O} (n^3)$ FIND REFERENCE the computation is much larger than it is for locally indexed methods.

| Name | Definition | Comment | AUC | Precision |
|------|------------|---------|-----|-----------|
|Katz  | $\mathbf{S} = \sum_{l=1}^{\infty} \beta^l \mathbf{A}^{l}$ | The score is based on the sum of the number of paths with length $l$ weighted by some factor, $\beta$ between 0 and 1. Shorter paths will have higher weights than longer paths using this scheme. | 0.488 | 0.789 |
|Leich, Holme, Newman (LHN2) | $\mathbf{S} = \psi (\mathbf{I} + \sum_{l=1}^{\infty} \phi^l \mathbf{A}^l)$| Similar to the Katz index, but this method chooses $\omega$ to be the inverse of the expected number of paths of any given path length | 0.511 | 0.408 |
|Average commute time | $ s_{x,y} = \frac{1}{l_{x,x} + l_{y,y} - 2 l_{x,y}}$ | Equal to the expected number of steps between any two nodes, $l_{x,y}$ is the (x,y)-th element in the pseudoinverse of the Laplacian matrix, $\mathbf{L}^{+}$ | 0.674 | 0.728 |

#### Katz indexing (KZI)
Katz indexing is a derivative of the common neighbors methodology which states that nodes are similar if there are more paths between them and this similarity decreases with the length of the path and for a very small $\beta$ Katz becomes similar to common neighbors [](#cite-KatzIndex). A closed form solution can be found from its explicit form as
$$
\mathbf{S} = \sum_{l=1}^{\infty} \beta^l \mathbf{A}^{l} = (\mathbf{I} - \beta \mathbf{A})^{-1} - \mathbf{I}
$$
The value of $\beta$ is a free parameter which can be optimized for.

#### Leicht, Holme, Newman (LHN2)
The LHN2 method is based on the assumption that nodes are similar if they have neighbors that are similar [](#cite-LHN2) and in its explicit form it is very similar to the Katz index except that it includes a self similarity measure in the form of $\psi$
$$
\mathbf{S} = \omega \mathbf{A} \mathbf{S} + \psi \mathbf{S} = \psi (\mathbf{I} + \sum_{l=1}^{\infty} \phi^l \mathbf{A}^l)
$$
Setting the constant $\psi$ to one this expression term by term states that every node has a similarity of one 1, $\mathbf{I}$, and with each node in its $l$-neighborhood it has a similarity of $\phi^l$.

#### Average Commute Time (ACM)

ACM is found as the average number of time steps a random walker would need to reach node $x$ from node $y$ and can be found from the pseudo-inverse of the Laplacian matrix of the graph [](#cite-Fouss06random-walkcomputation), $\mathbf{L} = \mathbf{D}-\mathbf{A}$ where $\mathbf{D}$ is a diagonal matrix containing the degrees of each node. From the pseudo-inverse of the Laplacian, $\mathbf{L}^+$ the average commute time is found as 
$$
\text{acm}_{x,y} = l_{x,x}^+ + l_{y,y}^+ - 2l_{x,y}^+
$$
To keep with the convention that higher scores are better the score is found as the inverse of the commute time, $s_{x,y} = \text{acm}_{x,y}^{-1}$.

### Quasi global indexing

Quasi global indices generally work on an $n$-neighborhood of a node instead of restricting themselves to immediate neighbors. This can give computational speed ups, but for small-world networks it can be just as fast to compute the closed form solutions of the global methods as $n$ increases (no reference, but it makes sense).

| Name | Definition | Comment | AUC | Precision |
|------|------------|---------|-----|-----------|
|Local Path Index | $\mathbf{S} = \sum_{i=2}^n \epsilon^{i-2} \mathbf{A}^i $| Similar to the Katz index for $n = \infty$ the computational complexity of this index is $\mathcal{O}(N\langle{k}\rangle^n)$ for a size N network with mean node degree of $\langle{k}\rangle$ | 0.790 | 0.568 |
|Local Random Walk | $s_{x,y} = q_{x} \pi_{x,y}(t) + q_{y} \pi_{y,x}(t) $| Equal to the probability of reaching node $y$ from node $x$ in t timesteps weighed by some initial configuration, $q$ indicating the probability of being at node $x$, summed by its mirror from $y$ to $x$. | N/I | N/I|
|Superimposed Random Walk | $s_{x,y} = \sum_{\tau}^{t} q_{x} \pi_{x,y}(\tau) + q_{y} \pi_{y,x}(\tau)$ | Similar to LRW, but the summation indicates a restart at each timestep | N/I | N/I | 

#### Local Path Indexing (LPI)

Local path indexing is the Katz index constrained to a neighborhood fixed to a size of $n$. This reduces the compute time as long as the average node degree is relatively small, but for small world networks with a small average shortest path length even small neighborhoods can quickly encompass the full set of nodes, making the computation of the inverse in the Katz index faster.

#### Local Random Walk (LRW)

The local random walk scores a pair of nodes, $x,y$, based on the probability of reaching node $y$ from node $x$ in a given number of timesteps, $t$ and vice versa. This requires that node $y$ is in the $t$-neighborhood of $x$ and penalizes nodes that have high degree nodes between them.

#### Superimposed Random Walk (SRW)

The superimposed random walk functions exactly like the LRW except that it restarts the random walker at every timestep, summing the probabilities for each increment of $\tau$.

# References

<a name="cite-Lu20111150"/><sup>[^](#ref-1) [^](#ref-2) [^](#ref-3) [^](#ref-4) </sup>Lü, Linyuan and Zhou, Tao. 2011. _Link prediction in complex networks: A survey _. [URL](http://www.sciencedirect.com/science/article/pii/S037843711000991X)

